In [13]:
import pandas as pd
from clingo import Control
import ast
from matching_utils import (
    get_invalid_matches, 
    check_mentor_capacity,
    check_day_conflicts,
    check_multiple_days
)

students_df = pd.read_csv('../DATASETS/studenten.csv')
mentors_df = pd.read_csv('../DATASETS/mentoren25.csv')
mentors2_df = pd.read_csv('../DATASETS/mentorenB.csv')  # Second mentor list

education_mapping = {
    'Associate': 1,
    'Bachelor': 2, 
    'Master': 3,
    'PhD': 4
}

n_mentors = 3  # Number of mentors from first list
m_mentors = 2  # Number of mentors from second list

def parse_literal_list(raw):
    if isinstance(raw, str):
        try:
            return ast.literal_eval(raw)
        except (ValueError, SyntaxError):
            return []
    return raw if isinstance(raw, list) else []

def normalize_token(text):
    if not isinstance(text, str):
        return ""
    return text.lower().replace(' ', '_')

students_cache = []
student_lookup = {}
for idx, row in students_df.iterrows():
    student_id = f"s{idx}"
    subject_atom = normalize_token(row['Onderwerp'])
    availability = [day.lower() for day in parse_literal_list(row['Beschikbaarheid'])]
    entry = {
        'id': student_id,
        'education_level': education_mapping[row['Opleidingsniveau']],
        'subject_atom': subject_atom,
        'availability': availability,
        'data': {
            'voornaam': row['Voornaam'],
            'achternaam': row['Achternaam'],
            'opleidingsniveau': row['Opleidingsniveau'],
            'onderwerp': row['Onderwerp'],
        },
    }
    students_cache.append(entry)
    student_lookup[student_id] = entry

mentors_cache = []
mentor_lookup = {}
# Process first mentor list
for idx, row in mentors_df.iterrows():
    mentor_id = f"m{idx}"
    subjects = [normalize_token(subject) for subject in parse_literal_list(row['Onderwerpen'])]
    availability = [day.lower() for day in parse_literal_list(row['Beschikbaarheid'])]
    entry = {
        'id': mentor_id,
        'mentor_type': 'type1',
        'education_level': education_mapping[row['Opleidingsniveau']],
        'subjects': subjects,
        'availability': availability,
        'max_students': row['Max_Studenten'],
        'data': {
            'voornaam': row['Voornaam'],
            'achternaam': row['Achternaam'],
            'opleidingsniveau': row['Opleidingsniveau'],
            'type': 'Type 1',
        },
    }
    mentors_cache.append(entry)
    mentor_lookup[mentor_id] = entry

# Process second mentor list
for idx, row in mentors2_df.iterrows():
    mentor_id = f"m2_{idx}"
    subjects = [normalize_token(subject) for subject in parse_literal_list(row['Onderwerpen'])]
    availability = [day.lower() for day in parse_literal_list(row['Beschikbaarheid'])]
    entry = {
        'id': mentor_id,
        'mentor_type': 'type2',
        'education_level': education_mapping[row['Opleidingsniveau']],
        'subjects': subjects,
        'availability': availability,
        'max_students': row['Max_Studenten'],
        'data': {
            'voornaam': row['Voornaam'],
            'achternaam': row['Achternaam'],
            'opleidingsniveau': row['Opleidingsniveau'],
            'type': 'Type 2',
        },
    }
    mentors_cache.append(entry)
    mentor_lookup[mentor_id] = entry

def generate_asp_facts():
    facts = []
    days = set()
    
    for student in students_cache:
        facts.append(f"student({student['id']}).")
        facts.append(f"education({student['id']}, {student['education_level']}).")
        facts.append(f"expertise({student['id']}, {student['subject_atom']}).")
        
        for day in student['availability']:
            facts.append(f"availability({student['id']}, {day}).")
            days.add(day)
    
    for mentor in mentors_cache:
        facts.append(f"mentor({mentor['id']}).")
        facts.append(f"mentor_type({mentor['id']}, {mentor['mentor_type']}).")
        facts.append(f"education({mentor['id']}, {mentor['education_level']}).")
        
        for subject in mentor['subjects']:
            facts.append(f"expertise({mentor['id']}, {subject}).")
        
        facts.append(f"max_students({mentor['id']}, {mentor['max_students']}).")
        
        for day in mentor['availability']:
            facts.append(f"availability({mentor['id']}, {day}).")
            days.add(day)
    
    for day in sorted(days):
        facts.append(f"day({day}).")
    
    return "\n".join(facts)

asp_facts = generate_asp_facts()

ctl = Control()
ctl.configuration.solve.models = 1
ctl.add("base", [], f"""
#const req_mentors_type1 = {n_mentors}.
#const req_mentors_type2 = {m_mentors}.

{asp_facts}

candidate(S, M, Day) :-
    student(S),
    mentor(M),
    expertise(S, Subj),
    expertise(M, Subj),
    education(S, ES),
    education(M, EM),
    EM > ES,
    availability(S, Day),
    availability(M, Day).

possible_day(S, Day) :- candidate(S, M, Day).

1 {{ match_day(S, Day) : possible_day(S, Day) }} 1 :- student(S).

{{ match(S, M, Day) : candidate(S, M, Day), mentor_type(M, type1) }} :- match_day(S, Day).
{{ match(S, M, Day) : candidate(S, M, Day), mentor_type(M, type2) }} :- match_day(S, Day).

% Exactly req_mentors_type1 mentors from type1
:- match_day(S, Day), #count {{ M : match(S, M, Day), mentor_type(M, type1) }} < req_mentors_type1.
:- match_day(S, Day), #count {{ M : match(S, M, Day), mentor_type(M, type1) }} > req_mentors_type1.

% Exactly req_mentors_type2 mentors from type2
:- match_day(S, Day), #count {{ M : match(S, M, Day), mentor_type(M, type2) }} < req_mentors_type2.
:- match_day(S, Day), #count {{ M : match(S, M, Day), mentor_type(M, type2) }} > req_mentors_type2.

:- match(S, M, Day), not match_day(S, Day).

:- mentor(M), max_students(M, Max), #count {{ S, Day : match(S, M, Day) }} > Max.

#maximize {{ 1, S : match_day(S, Day) }}.

#show match/3.

""")

ctl.ground([("base", [])])

def make_matches():
    matches = []

    def collect_matches(model):
        nonlocal matches
        grouped = {}

        for symbol in model.symbols(shown=True):
            if symbol.name == "match":
                student_id = symbol.arguments[0].name
                mentor_id = symbol.arguments[1].name
                day = symbol.arguments[2].name

                key = (student_id, day)
                grouped.setdefault(key, {'type1': [], 'type2': []})
                
                mentor_entry = mentor_lookup[mentor_id]
                mentor_type = mentor_entry['mentor_type']
                grouped[key][mentor_type].append(mentor_id)

        matches = []
        for (student_id, day), mentor_groups in grouped.items():
            student_entry = student_lookup[student_id]
            student_data = student_entry['data'].copy()

            mentors_type1 = []
            for mentor_id in sorted(mentor_groups['type1']):
                mentor_entry = mentor_lookup[mentor_id]
                mentors_type1.append(mentor_entry['data'].copy())

            mentors_type2 = []
            for mentor_id in sorted(mentor_groups['type2']):
                mentor_entry = mentor_lookup[mentor_id]
                mentors_type2.append(mentor_entry['data'].copy())

            matches.append((student_data, mentors_type1, mentors_type2, day))

    result = ctl.solve(on_model=collect_matches)
    return matches if result.satisfiable else []



# Usage
matches = make_matches()
print(f"Total matched students: {len(matches)} (each with {n_mentors} type1 mentors and {m_mentors} type2 mentors)")

def export_matches_to_csv(matches, filename='../DATASETS/matches.csv'):
    if not matches:
        print("No matches to export")
        return

    rows = []
    for student, mentors_type1, mentors_type2, day in matches:
        row = {
            'Student': f"{student['voornaam']} {student['achternaam']}",
            'Day': day.capitalize(),
            'Mentors_Type1': "; ".join(f"{m['voornaam']} {m['achternaam']}" for m in mentors_type1),
            'Mentors_Type2': "; ".join(f"{m['voornaam']} {m['achternaam']}" for m in mentors_type2),
        }

        rows.append(row)

    matches_df = pd.DataFrame(rows)
    matches_df.to_csv(filename, index=False)
    print(f"Exported {len(matches)} matches to {filename}")
    return matches_df

# Export the matches
matches_df = export_matches_to_csv(matches)

if matches_df is not None:
    print("\nPreview of exported matches:")
    print(matches_df.head())


Total matched students: 0 (each with 3 type1 mentors and 2 type2 mentors)
No matches to export


In [ ]:
def diagnose_matching_problem():
    """Diagnose why no matches are being found"""
    print("=== Diagnostic Information ===\n")
    
    # Check students
    print(f"Total students: {len(students_cache)}")
    if students_cache:
        sample = students_cache[0]
        print(f"Sample student: {sample['data']['voornaam']} {sample['data']['achternaam']}")
        print(f"  - Subject: {sample['subject_atom']}")
        print(f"  - Level: {sample['education_level']}")
        print(f"  - Availability: {sample['availability']}")
    
    # Check mentors by type
    type1_mentors = [m for m in mentors_cache if m['mentor_type'] == 'type1']
    type2_mentors = [m for m in mentors_cache if m['mentor_type'] == 'type2']
    
    print(f"\nTotal type1 mentors: {len(type1_mentors)}")
    print(f"Total type2 mentors: {len(type2_mentors)}")
    
    if type1_mentors:
        sample = type1_mentors[0]
        print(f"\nSample type1 mentor: {sample['data']['voornaam']} {sample['data']['achternaam']}")
        print(f"  - Subjects: {sample['subjects']}")
        print(f"  - Level: {sample['education_level']}")
        print(f"  - Availability: {sample['availability']}")
        print(f"  - Max students: {sample['max_students']}")
    
    if type2_mentors:
        sample = type2_mentors[0]
        print(f"\nSample type2 mentor: {sample['data']['voornaam']} {sample['data']['achternaam']}")
        print(f"  - Subjects: {sample['subjects']}")
        print(f"  - Level: {sample['education_level']}")
        print(f"  - Availability: {sample['availability']}")
        print(f"  - Max students: {sample['max_students']}")
    
    # Check for potential candidates
    print("\n=== Checking Candidate Availability ===")
    
    candidates_type1 = 0
    candidates_type2 = 0
    
    for student in students_cache:
        student_candidates_type1 = []
        student_candidates_type2 = []
        
        for mentor in mentors_cache:
            # Check basic matching conditions
            if mentor['education_level'] > student['education_level']:
                if student['subject_atom'] in mentor['subjects']:
                    common_days = set(student['availability']) & set(mentor['availability'])
                    if common_days:
                        if mentor['mentor_type'] == 'type1':
                            student_candidates_type1.append((mentor, common_days))
                        else:
                            student_candidates_type2.append((mentor, common_days))
        
        candidates_type1 += len(student_candidates_type1)
        candidates_type2 += len(student_candidates_type2)
        
        if len(student_candidates_type1) < n_mentors or len(student_candidates_type2) < m_mentors:
            print(f"\n[ISSUE] Student {student['data']['voornaam']} {student['data']['achternaam']}:")
            print(f"  - Needs {n_mentors} type1 mentors, found {len(student_candidates_type1)} candidates")
            print(f"  - Needs {m_mentors} type2 mentors, found {len(student_candidates_type2)} candidates")
            if len(student_candidates_type1) + len(student_candidates_type2) > 0:
                print(f"  - Subject: {student['subject_atom']}, Level: {student['education_level']}")
    
    print(f"\nTotal type1 candidate matches possible: {candidates_type1}")
    print(f"Total type2 candidate matches possible: {candidates_type2}")
    print(f"\nRequired per student: {n_mentors} type1 + {m_mentors} type2 = {n_mentors + m_mentors} total")

# Run diagnostics
diagnose_matching_problem()

In [14]:
def debug_matching():
    matches = make_matches()
    print(f"Total matches found: {len(matches)}")
    
    if matches:
        print("\nSample matches:")
        for student, mentors_type1, mentors_type2, day in matches[:5]:
            print(f"  {student['voornaam']} {student['achternaam']} ({student['opleidingsniveau']}, {student['onderwerp']})")
            print(f"  Day: {day.capitalize()}")
            print(f"  Type 1 Mentors:")
            for mentor in mentors_type1:
                print(f"    -> {mentor['voornaam']} {mentor['achternaam']} ({mentor['opleidingsniveau']})")
            print(f"  Type 2 Mentors:")
            for mentor in mentors_type2:
                print(f"    -> {mentor['voornaam']} {mentor['achternaam']} ({mentor['opleidingsniveau']})")
            print()
    
    # Flatten matches for validation (combine both mentor types)
    flat_matches = [
        (student, mentor, day)
        for student, mentors_type1, mentors_type2, day in matches
        for mentor in (mentors_type1 + mentors_type2)
    ]
    
    print("\n=== Validation Results ===")
    
    invalid_matches = get_invalid_matches(flat_matches)
    if invalid_matches:
        print(f"[FAIL] Invalid matches: {len(invalid_matches)}")
        for student, mentor, day in invalid_matches[:3]:
            print(f"   - {student['voornaam']} {student['achternaam']} -> {mentor['voornaam']} {mentor['achternaam']} on {day}")
    else:
        print("[PASS] All matches are valid")
    
    over_capacity = check_mentor_capacity(flat_matches)
    if over_capacity:
        print(f"[FAIL] Mentors over capacity:")
        for item in over_capacity:
            print(f"   - {item['mentor']}: {item['matched']} matched (max: {item['max']})")
    else:
        print("[PASS] All mentors within capacity")
    
    day_conflicts = check_day_conflicts(flat_matches)
    if day_conflicts:
        print(f"[FAIL] Day conflicts found:")
        for conflict in day_conflicts:
            print(f"   - {conflict['type'].capitalize()} {conflict['name']} has {conflict['count']} matches on {conflict['day']}")
    else:
        print("[PASS] No day conflicts")
    
    multiple_days = check_multiple_days(flat_matches)
    if multiple_days:
        print(f"[FAIL] Students matched on multiple days:")
        for item in multiple_days:
            print(f"   - {item['student']}: {', '.join(item['days'])}")
    else:
        print("[PASS] No students matched on multiple days")

debug_matching()


Total matches found: 0

=== Validation Results ===
[PASS] All matches are valid
[PASS] All mentors within capacity
[PASS] No day conflicts
[PASS] No students matched on multiple days
